<a href="https://colab.research.google.com/github/Milonyash/testFastRCNN/blob/master/Simple_RCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**1. Selective Search**


In [2]:
!pip install selectivesearch

  Stored in directory: /root/.cache/pip/wheels/ab/a0/bd/a56e017d10730401fce75f62ff9364004368b0b96f12c026ba
Successfully built selectivesearch


In [0]:
import skimage.data
from skimage import io
import skimage.transform as T
import skimage.util as util
import selectivesearch
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

IMG_HEIGHT = 224
IMG_WIDTH = 224

def get_regions_propsal(imgpath):
    #reading image from file
    img = io.imread(imgpath)
    img = T.resize(img,(IMG_HEIGHT,IMG_WIDTH,3),mode='reflect')
    
    
    #get region proposal using selective search
    img_lbl, regions = selectivesearch.selective_search(img, scale=500, sigma=0.9, min_size=100)
    
    #show the original image
    plt.imshow(img)
    plt.show()
    
    print("There is %d regions proposed"%len(regions))
    
    #crop the original image according to the regions proposed and store it to a new array
    regions_proposal=list()
    for r in regions:
        rect = r['rect']
        y1 = rect[0]
        y2 = rect[0] + rect[2]
        x1 = rect[1]
        x2 = rect[1] + rect[3]
        cropped_im = img[x1:x2,y1:y2]
        #this is problematic
        #cropped_im = T.resize(cropped_im,(IMG_HEIGHT,IMG_WIDTH,3),mode='constant')
        regions_proposal += [(cropped_im,rect)]
    
    return regions_proposal,regions

In [0]:
#Результаты
'''
import warnings

reg_prop,raw_regions = get_regions_propsal('test.jpg')
regions = list()

for i,rp in enumerate(reg_prop):
    #continue;
    image = T.resize(rp[0],(IMG_HEIGHT,IMG_WIDTH,3),mode='constant')
    path = "img/%d__%s.jpg"%(i,rp[1])
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        io.imsave(path,image)
    regions += [{"num":i,"filepath":path,"rect":rp[1]}]
    
print("All image regions have been created")
'''

In [0]:
def ShowRegionProposal(reg):  
    img = io.imread('test.jpg')
    img = T.resize(img,(IMG_HEIGHT,IMG_WIDTH,3),mode='reflect')
    candidates = set()
    for r in reg:
    # excluding same rectangle (with different segments)
        if r['rect'] in candidates:
            continue
        # excluding regions smaller than 2000 pixels
        if r['size'] < 2000:
            continue
        # distorted rects
        x, y, w, h = r['rect']
        if w / h > 1.2 or h / w > 1.2:
            continue
        candidates.add(r['rect'])

        # draw rectangles on the original image
        fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(6, 6))
        ax.imshow(img)
        for x, y, w, h in candidates:
            #print(x, y, w, h)
            rect = mpatches.Rectangle(
                (x, y), w, h, fill=False, edgecolor='red', linewidth=1)
            ax.add_patch(rect)
    plt.show()
    

def displayRegion(r,label,prob):
            #reading image from file
        img = io.imread('images/test.jpg')
        img = T.resize(img,(IMG_HEIGHT,IMG_WIDTH,3),mode='reflect')
    
        # distorted rects
        x, y, w, h = r
        # draw rectangles on the original image
        
        fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(6, 6))
        ax.imshow(img)
        rect = mpatches.Rectangle((x, y), w, h, fill=False, edgecolor='red', linewidth=1)
        ax.add_patch(rect)
        plt.title("%s: p %.2f"%(label,prob));
        plt.show()
        
        

In [0]:
ShowRegionProposal(raw_regions)

In [0]:

img= io.imread('img/72.jpg')
plt.imshow(img)
plt.show()


In [0]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.optimizers import SGD
import numpy as np

from keras.applications.resnet50 import ResNet50
#Load ResNet50 architecture & its weights
modelRestNet50 = ResNet50(include_top=True, weights='imagenet')

In [5]:
!wget http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar

--2019-05-03 09:43:02--  http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
Resolving host.robots.ox.ac.uk (host.robots.ox.ac.uk)... 129.67.94.152
Connecting to host.robots.ox.ac.uk (host.robots.ox.ac.uk)|129.67.94.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 460032000 (439M) [application/x-tar]
Saving to: ‘VOCtrainval_06-Nov-2007.tar’

VOCtrainval_06-Nov- 100%[===================>] 438.72M  9.56MB/s    in 49s     

2019-05-03 09:43:52 (8.95 MB/s) - ‘VOCtrainval_06-Nov-2007.tar’ saved [460032000/460032000]



In [6]:
!tar -xvf VOCtrainval_06-Nov-2007.tar

VOCdevkit/
VOCdevkit/VOC2007/
VOCdevkit/VOC2007/Annotations/
VOCdevkit/VOC2007/Annotations/000005.xml
VOCdevkit/VOC2007/Annotations/000007.xml
VOCdevkit/VOC2007/Annotations/000009.xml
VOCdevkit/VOC2007/Annotations/000012.xml
VOCdevkit/VOC2007/Annotations/000016.xml
VOCdevkit/VOC2007/Annotations/000017.xml
VOCdevkit/VOC2007/Annotations/000019.xml
VOCdevkit/VOC2007/Annotations/000020.xml
VOCdevkit/VOC2007/Annotations/000021.xml
VOCdevkit/VOC2007/Annotations/000023.xml
VOCdevkit/VOC2007/Annotations/000024.xml
VOCdevkit/VOC2007/Annotations/000026.xml
VOCdevkit/VOC2007/Annotations/000030.xml
VOCdevkit/VOC2007/Annotations/000032.xml
VOCdevkit/VOC2007/Annotations/000033.xml
VOCdevkit/VOC2007/Annotations/000034.xml
VOCdevkit/VOC2007/Annotations/000035.xml
VOCdevkit/VOC2007/Annotations/000036.xml
VOCdevkit/VOC2007/Annotations/000039.xml
VOCdevkit/VOC2007/Annotations/000041.xml
VOCdevkit/VOC2007/Annotations/000042.xml
VOCdevkit/VOC2007/Annotations/000044.xml
VOCdevkit/VOC2007/Annotations/000046.

In [0]:
import os
import random
import numpy as np
import skimage.data
from skimage import io
import skimage.transform as T
import skimage.util as util
import selectivesearch
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input

#for Intersection over Union
from collections import namedtuple
import cv2

import warnings

import xml.etree.ElementTree as ET

#SELECTIVE SEARCH PARAMS
SCALE = 500
SIGMA = 0.2
MIN_SIZE = 225

UoI_THRESHOLD = 0.5

#random.seed(2506)

IMG_HEIGHT = 224
IMG_WIDTH = 224

IMAGES_DIR = 'JPEGImages'
LABELS_DIR = 'ImageSets\\Main'
ANNOTATIONS_DIR = 'Annotations'


EXT = '.jpg'
XML = '.xml'

LABELS = ['aeroplane', 'bicycle', 'bird', 'boat',
           'bottle', 'bus', 'car', 'cat', 'chair',
           'cow', 'diningtable', 'dog', 'horse',
           'motorbike', 'person', 'pottedplant',
           'sheep', 'sofa', 'train', 'tvmonitor']
#LABELS += 'background'

NBCLASSES = len(LABELS)



class DataGen(object): #PascalVOCDataGeneratorForObjectDetection
    

    def __enter__(self):
            return self # this is bound to the `as` part


    def __exit__(self, *err):
        return
    
    
    def __init__(self, subset, data_path,keep = 2):
        
        self.keep = keep
        
        assert subset in ['train', 'val', 'trainval', 'test']
        self.subset = subset

        self.data_path = data_path
        self.images_path = os.path.join(self.data_path, 'JPEGImages')
        self.labels_path = os.path.join(self.data_path, 'ImageSets', 'Main')
        
        # The id_to_label dict has the following structure
        # key : image's id (e.g. 00084)
        # value : image's label (e.g. [0, 0, 1, 0, ..., 1, 0])
        self.id_to_label = {}

        self.labels = LABELS
        self.nb_classes = len(self.labels) # 20 classes for PascalVOC

        # Get all the images' ids for the given subset
        self.images_ids_in_subset = self._get_images_ids_from_subset(self.subset)
        self.nb_images = len(self.images_ids_in_subset)
        
        self.DataSet = np.asarray(self.images_ids_in_subset).astype(int);
        
        return
        #public acces dataset and according dict {image:nb_regions}
        #Get a dict of each image and its number of regions
        self.image_dict,self.total_nb_images = self._get_region_image_table(6403);
        dic  = {k:v for (k,v) in self.image_dict.items() if k in self.DataSet}
        self.DataDict = dic        
        self.TotalDataSet = sum(dic.values())


    def generateRegionsData(self,Display = False,from_image=0,to_image=1): #Be very careful here
        
        with open('annotations/labels_%06d_%06d.txt'%(from_image+1,to_image+1), 'w'): pass
        for i in range(from_image+1,to_image+1):
            imagePath = self._get_image_file_path(i)
    
            bb_labels,bb_boxes = self._get_ground_truth_bb(i)
            reg_prop,raw_regions = self._get_regions_propsal(imagePath)        
            regions = []
            labels = []
            #Compare every proposed region with defined Pascal VOC regions
            for r,rp in enumerate(reg_prop):
                saved = False
                for l,bb in enumerate(bb_boxes):
                    pred = self._rect_to_bbox(rp[1])
                    gt = bb;
                    UoI = self._bb_intersection_over_union(gt,pred)
                    if(UoI>UoI_THRESHOLD):               
                        if (Display): self._display_UiO(imagePath,gt,pred)
                        if not saved:
                            regions += [rp]
                            labels += [LABELS.index(bb_labels[l])]
                            saved = True
                        else:
                            labels[-1]= LABELS.index(bb_labels[l])
                    else:
                        if not saved:
                            regions += [rp]
                            labels += [20]
                            saved = True
            print("Treating image N°: %06d ..."%i)
            print("\t There is %d regions proposed"%(len(raw_regions),))
            
            for j,rp in enumerate(regions):
                #continue;
                #image = T.resize(rp[0],(IMG_HEIGHT,IMG_WIDTH,3),mode='constant')
                image = rp[0]
                path = "images/%06d_%03d.jpg"%(i,j+1)
                #path = "images/%03d_%s.jpg"%(j,rp[1])
                with warnings.catch_warnings():
                    warnings.simplefilter("ignore")
                    try:
                        io.imsave(path,image)
                        #regions += [{"num":i,"filepath":path,"rect":rp[1]}]
                        with open('annotations/labels_%06d_%06d.txt'%(from_image+1,to_image+1), 'a') as f:
                            f.write("%06d %d %d\n"%(i,j+1,labels[j]))
                    except Exception as e:
                        print(repr(e))
                        pass
            print("\t %d regions were saved"%(len(regions),))
              
        print("Generation Completed");
        
        
    def generate_xy_from_image(self,image_id):    
        regions,labels = self._generate_regions_of_image(image_id)
        X_batch = []
        Y_batch = []
        for i,region in enumerate(regions):
            try:
                img = T.resize(region, (IMG_HEIGHT,IMG_WIDTH),mode="constant")
                X_batch.append(img)       
                y = self._label_to_one_hot(labels[i])
                Y_batch.append(y)                    
            except Exception as e:
                continue
        Y_batch = np.reshape(Y_batch, (-1, self.nb_classes+1))
        X_batch = np.reshape(X_batch, (-1, IMG_HEIGHT, IMG_WIDTH, 3))
        X_batch = preprocess_input(X_batch, data_format='channels_last')
        
        return X_batch,Y_batch  
        
    def _generate_regions_of_image(self, imageId,Display=False):       
        imagePath = self._get_image_file_path(imageId)

        bb_labels,bb_boxes = self._get_ground_truth_bb(imageId)
        reg_prop,raw_regions = self._get_regions_propsal(imagePath)        

        regions = []
        labels = []
        #Compare every proposed region with defined Pascal VOC regions
        for r,rp in enumerate(reg_prop):
            saved = False
            for l,bb in enumerate(bb_boxes):
                pred = self._rect_to_bbox(rp[1])
                gt = bb;
                UoI = self._bb_intersection_over_union(gt,pred)
                if(UoI>UoI_THRESHOLD):               
                    if (Display): self._display_UiO(imagePath,gt,pred)
                    if not saved:
                        regions.append(rp[0])
                        labels.append(LABELS.index(bb_labels[l]))
                        saved = True
                    else:
                        labels[-1]= LABELS.index(bb_labels[l])
                else:
                    if not saved:
                        regions.append(rp[0])
                        labels.append(20)
                        saved = True
        
        return regions,labels

    def _get_ground_truth_bb(self,fileNum):
        filePath = self._get_annotation_file_path(fileNum)
        tree = ET.parse(filePath)
        root = tree.getroot()
        
        labels = []
        bboxes = []

        for child in root:
            if child.tag == 'object':
                labels += [child[0].text]
                for subchild in child:
                    if subchild.tag == 'bndbox':
                        bbox = subchild
                        bboxes += [(int(bbox[0].text),int(bbox[1].text),int(bbox[2].text),int(bbox[3].text))]
        return labels,bboxes

    def _bbox_to_rect(self,rect):
        y = rect[0]
        x = rect[1]
        
        w = rect[2] - y
        h = rect[3] - x
        
        return (x,y,w,h)

    def _rect_to_bbox(self,rect):
        xmin = rect[0]
        ymin = rect[1]
        xmax = rect[0] + rect[2]
        ymax = rect[1] + rect[3]
        
        return (xmin,ymin,xmax,ymax)
    
    
    def _display_UiO(self,imagePath,gt,pred):
        
        image = cv2.imread(imagePath)
 
        # draw the ground-truth bounding box along with the predicted
        # bounding box
        
        cv2.rectangle(image, tuple([gt[0],gt[1]]), 
            tuple([gt[2],gt[3]]), (0, 255, 0), 2)
        cv2.rectangle(image, tuple([pred[0],pred[1]]), 
            tuple([pred[2],pred[3]]), (0, 0, 255), 2)

        # compute the intersection over union and display it
        iou = self._bb_intersection_over_union(gt,pred)
        cv2.putText(image, "IoU: {:.14f}".format(iou), (10, 30),cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        #print("{}: {:.4f}".format(detection.image_path, iou))
        # show the output image
        cv2.imshow("Image", image)
        cv2.waitKey(0)
    
  
    
    def _get_regions_propsal(self,imgpath):
        #reading image from file
        img = io.imread(imgpath)
        #img = T.resize(img,(IMG_HEIGHT,IMG_WIDTH,3),mode='reflect')


        #get region proposal using selective search
        img_lbl, regions = selectivesearch.selective_search(img, scale=SCALE, sigma=SIGMA, min_size=MIN_SIZE)


        #crop the original image according to the regions proposed and store it to a new array
        regions_proposal=list()
        for r in regions:
            rect = r['rect']
            y1 = rect[0]
            y2 = rect[0] + rect[2]
            x1 = rect[1]
            x2 = rect[1] + rect[3]
            cropped_im = img[x1:x2,y1:y2]
            #this is problematic
            #cropped_im = T.resize(cropped_im,(IMG_HEIGHT,IMG_WIDTH,3),mode='constant')
            regions_proposal += [(cropped_im,rect)]
    
        return regions_proposal,regions
    

    def _bb_intersection_over_union(self,boxA, boxB):

        # determine the (x, y)-coordinates of the intersection rectangle
        xA = max(boxA[0], boxB[0])
        yA = max(boxA[1], boxB[1])
        xB = min(boxA[2], boxB[2])
        yB = min(boxA[3], boxB[3])

        # compute the area of intersection rectangle
        #interArea = (xB - xA + 1) * (yB - yA + 1)
        interArea = max(0,(xB - xA + 1)) * max(0,(yB - yA + 1)) #correction

        # compute the area of both the prediction and ground-truth
        # rectangles
        boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
        boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
        
       

        # compute the intersection over union by taking the intersection
        # area and dividing it by the sum of prediction + ground-truth
        # areas - the interesection area
        iou = interArea / float(boxAArea + boxBArea - interArea)

        # return the intersection over union value
        return iou
    
    

    def _get_image_file_path(self,num):
    
        imagesPath = os.path.join(self.data_path,IMAGES_DIR)
            
        imageNumber = num;
        imageName = "%06d" % (imageNumber,)
        imageFile = '%s%s'%(imageName,EXT)

        imageFilePath = os.path.join(imagesPath,imageFile)
    
        return imageFilePath;

    def _get_annotation_file_path(self,num):
        dirPath = os.path.join(self.data_path,ANNOTATIONS_DIR)
            
        fileNumber = num;
        fileName = "%06d" % (fileNumber,)
        annotationFile = '%s%s'%(fileName,XML)

        annotationFilePath = os.path.join(dirPath,annotationFile)
        
        return annotationFilePath
    
    def _get_total_number_images(self):
        with open('annotations/labels.txt','r') as f:
            l = f.read().splitlines()
            return len(l)
        
    def _get_regions_of_image(self, imageId):
        filePath = 'annotations/labels.txt'
        labels = np.loadtxt(filePath).astype(int)
        imageLines = np.where(labels[:,0]==int(imageId))[0]
        regions=[]
        for line in labels[imageLines]:
            regions += [ ["%06d_%03d.jpg"%(line[0],line[1]),line[2]]]
        return regions        
        
    def _id_to_label_dict(self):
        for image_id in self.images_ids_in_subset:
            self.id_to_label[image_id] = np.zeros(self.nb_classes)
        pass
        
    def _label_to_one_hot(self,label):
        label_one_hot = np.zeros(self.nb_classes+1)
        label_one_hot[label]=1;
        return label_one_hot.astype(int);
        
    def _initialize_id_to_label_dict(self):
        for image_id in self.images_ids_in_subset:
            self.id_to_label[image_id] = np.zeros(self.nb_classes)

    def _fill_id_to_label_dict_with_classes(self):
        pass
        """_fill_id_to_label_dict_with_classes
        For each class, the <class>_<subset>.txt file contain the presence information
        of this class in the image
        """
        for i in range(self.nb_classes):
            label = self.labels[i]
            # Open the <class>_<subset>.txt file
            with open(os.path.join(self.labels_path, "%s_%s.txt" % (label, self.subset)), 'r') as f:
                lines = f.read().splitlines()
                for line in lines:
                    splited_line = line.split()
                    image_id = splited_line[0]
                    is_present = int(splited_line[1])
                    if is_present == 1:
                        self.id_to_label[image_id][i] = 1

    def _get_region_image_table(self,last):
        fileLines = np.loadtxt('annotations/labels.txt')
        nbr={}
        for i in range(1,last):
            regions =  np.where(fileLines[:,0]==i)[0]
            nbr[i]=len(regions)
        return nbr,len(fileLines)
    
    
    
    def _get_images_ids_from_subset(self, subset):
        """_get_images_ids_from_subset
        The images' ids are found in the <subset>.txt file in ImageSets/Main
        """
        with open(os.path.join(self.labels_path, subset + '.txt'), 'r') as f:
            images_ids = f.read().splitlines()
            random.shuffle(images_ids)
        return images_ids[:self.keep]
    
    
    def get_xy_from_image(self,iid):
        image_id = "%06d"%iid
        img = image.load_img(os.path.join(self.images_path, image_id + EXT), grayscale=False, target_size=(IMG_HEIGHT, IMG_WIDTH))         
        #treat regions instead of whole images
        regions_labels = self._get_regions_of_image(image_id)
        X_batch = []
        Y_batch = []
        for region in regions_labels:
                imgPath = os.path.join('images', region[0])
                img = image.load_img(imgPath, grayscale=False, target_size=(IMG_HEIGHT, IMG_WIDTH))       
                # Cast the Image object to a numpy array and put the channel has the last dimension
                img_arr = image.img_to_array(img, data_format='channels_last')
                X_batch.append(img_arr)       
                y = self._label_to_one_hot(region[1])
                Y_batch.append(y)                    
        # resize X_batch in (batch_size, IMG_HEIGHT, IMG_WIDTH, 3) 
        X_batch = np.reshape(X_batch, (-1, IMG_HEIGHT, IMG_WIDTH, 3))
        # resize Y_batch in (None, nb_classes) 
        Y_batch = np.reshape(Y_batch, (-1, self.nb_classes+1))
        # The preprocess consists of substracting the ImageNet RGB means values
        # https://github.com/keras-team/keras/blob/master/keras/applications/imagenet_utils.py#L72
        X_batch = preprocess_input(X_batch, data_format='channels_last') 
        return X_batch,Y_batch     
    
    
    def bias_flow(self,X,Y,batch_size=128,pos_size=32,full_batch=1):

        nb_batches = int(len(Y) / batch_size) + 1
        pos_prop = pos_size
        neg_prop = batch_size-pos_prop
        
        maxes = np.argmax(Y,axis=1)
        #positive windows
        pos = np.where(maxes!=20)[0]
        #negative windows
        neg = np.where(maxes==20)[0]
        
        #separate positive and negative images
        posX = X[pos]
        posY = Y[pos]
        
        posEx = posX,posY
        
        negX = X[neg]
        negY = Y[neg]
        
        negEx = negX,negY
        
        posInd = np.arange(len(pos))
        negInd = np.arange(len(neg))
        
        np.random.shuffle(posInd)
        np.random.shuffle(negInd)
        
        pos_from = 0
        pos_to = pos_prop

        neg_from = 0
        neg_to = neg_prop
        
        allposInd = posInd
        allnegInd = negInd
        
        while True:          
            for i in range(nb_batches):
 
                if full_batch==1:  
                    #shuffle indices
                    np.random.shuffle(allposInd)
                    np.random.shuffle(allnegInd)
                    
                    notAll = True
                    posClass = np.full((pos_prop),-1)
                    posInd =np.arange(pos_prop)
                    i = 0
                    j = 0
                    while notAll:
                        c = np.argmax(posY[allposInd[i]])+1
                        if (np.all(posClass != c) or len(np.unique(np.argmax(posY[posInd],axis=1)))>20):
                            posClass[j] = c
                            posInd[j] = allposInd[i]
                            j+=1
                        i+=1;
                        notAll = j<pos_prop and i<len(allposInd)
                    negInd = negInd[:neg_prop]
                    
                    #try to pass on all the examples
                elif full_batch == 2:   
                    
                    posInd = allposInd[pos_from:pos_to] 
                    negInd = allnegInd[neg_from:neg_to]
                    
                    pos_from += pos_prop
                    pos_to += pos_prop
                    
                    neg_from += neg_prop
                    neg_to +=neg_prop
                    
                    if(pos_to)>=len(pos):
                        pos_from = 0
                        pos_to = pos_prop
                        #np.random.shuffle(allposInd)
                        
                    if(neg_to)>=len(neg):
                        neg_from = 0
                        neg_to = neg_prop
                        #np.random.shuffle(allnegInd)                        
                else:
                    #totaly random 
                    
                    #shuffle indices
                    np.random.shuffle(posInd)
                    np.random.shuffle(negInd)
                    posInd = posInd[:pos_prop] 
                    negInd = negInd[:neg_prop]                     

                #mini batch of 128 (32 positive windows + 96 negative windows)
                X_batch = np.concatenate((posX[posInd],negX[negInd]),axis = 0)
                Y_batch = np.concatenate((posY[posInd],negY[negInd]),axis = 0)
                yield(X_batch, Y_batch)
                
                
    
    def _generate_batch(self, batch_size=32):
        nb_batches = int(len(self.images_ids_in_subset) / batch_size) + 1
        nb_batches = 1
        size = len(self.images_ids_in_subset)
        # Before each epoch we shuffle the images' ids
        random.shuffle(self.images_ids_in_subset)
        for i in range(nb_batches):
                # We first get all the images' ids for the next batch
                current_bach = self.images_ids_in_subset[i*batch_size:(i+1)*size]
                #X_batch = []
                #Y_batch = []
                for image_id in current_bach:
                    print(image_id)
                    # Load the image and resize it. We get a PIL Image object 
                    img = image.load_img(os.path.join(self.images_path, image_id + EXT), grayscale=False, target_size=(IMG_HEIGHT, IMG_WIDTH))      
                    
                    #treat regions instead of whole images
                    regions = self._get_regions_of_image(image_id)
                    nb_mini_batches = int(len(regions) / batch_size) + 1
                    for j in range(nb_mini_batches):
                        current_mini_batch = regions[j*batch_size:(j+1)*batch_size]
                        X_batch = []
                        Y_batch = []
                        for region in current_mini_batch:
                            imgPath = os.path.join('images', region[0])
                            img = image.load_img(imgPath, grayscale=False, target_size=(IMG_HEIGHT, IMG_WIDTH))       
                            # Cast the Image object to a numpy array and put the channel has the last dimension
                            img_arr = image.img_to_array(img, data_format='channels_last')
                            X_batch.append(img_arr)       
                            y = self._label_to_one_hot(region[1])
                            Y_batch.append(y)
                    
                        # resize X_batch in (batch_size, IMG_HEIGHT, IMG_WIDTH, 3) 
                        X_batch = np.reshape(X_batch, (-1, IMG_HEIGHT, IMG_WIDTH, 3))
                        # resize Y_batch in (None, nb_classes) 
                        Y_batch = np.reshape(Y_batch, (-1, self.nb_classes+1))
                        # The preprocess consists of substracting the ImageNet RGB means values
                        # https://github.com/keras-team/keras/blob/master/keras/applications/imagenet_utils.py#L72
                        X_batch = preprocess_input(X_batch, data_format='channels_last') 
                        print(X_batch.shape)
                        print(Y_batch.shape)
    
    def flow2(self, batch_size=32):
        """flow
        This is a generator which load the images and preprocess them on the fly
        When calling next python build in function, it returns a batch with a given size
        with a X_batch of size (None, IMG_HEIGHT, IMG_WIDTH, 3)
        and a Y_batch of size (None, nb_classes)
        The first dimension is the batch_size if there is enough images left otherwise 
        it will be less
        :param batch_size: the batch's size
        """
        nb_batches = int(len(self.images_ids_in_subset) / batch_size) + 1
        
        nb_batches = 1
        size = len(self.images_ids_in_subset)
        
        while True:
            # Before each epoch we shuffle the images' ids
            random.shuffle(self.images_ids_in_subset)
            for i in range(nb_batches):
                # We first get all the images' ids for the next batch
                current_bach = self.images_ids_in_subset[i*batch_size:(i+1)*size]
                #X_batch = []
                #Y_batch = []
                for image_id in current_bach:
                    # Load the image and resize it. We get a PIL Image object 
                    img = image.load_img(os.path.join(self.images_path, image_id+EXT),grayscale=False,target_size=(IMG_HEIGHT, IMG_WIDTH))      
                    #treat regions instead of whole images
                    regions = self._get_regions_of_image(image_id)
                    nb_mini_batches = int(len(regions) / batch_size) + 1
                    for j in range(nb_mini_batches):
                        current_mini_batch = regions[j*batch_size:(j+1)*batch_size]
                        X_batch = []
                        Y_batch = []
                        for region in current_mini_batch:
                            imgPath = os.path.join('images', region[0])
                            img = image.load_img(imgPath, grayscale=False, target_size=(IMG_HEIGHT, IMG_WIDTH))       
                            # Cast the Image object to a numpy array and put the channel has the last dimension
                            img_arr = image.img_to_array(img, data_format='channels_last')
                            X_batch.append(img_arr)       
                            y = self._label_to_one_hot(region[1])
                            Y_batch.append(y)
                    
                        # resize X_batch in (batch_size, IMG_HEIGHT, IMG_WIDTH, 3) 
                        X_batch = np.reshape(X_batch, (-1, IMG_HEIGHT, IMG_WIDTH, 3))
                        # resize Y_batch in (None, nb_classes) 
                        Y_batch = np.reshape(Y_batch, (-1, self.nb_classes+1))
                        # The preprocess consists of substracting the ImageNet RGB means values
                        # https://github.com/keras-team/keras/blob/master/keras/applications/imagenet_utils.py#L72
                        X_batch = preprocess_input(X_batch, data_format='channels_last')
                        if(len(X_batch)>0):
                            yield(X_batch, Y_batch)    

                
    def flow(self, batch_size=32):
        """flow
        This is a generator which load the images and preprocess them on the fly
        When calling next python build in function, it returns a batch with a given size
        with a X_batch of size (None, IMG_HEIGHT, IMG_WIDTH, 3)
        and a Y_batch of size (None, nb_classes)
        The first dimension is the batch_size if there is enough images left otherwise 
        it will be less
        :param batch_size: the batch's size
        """
        nb_batches = int(len(self.images_ids_in_subset) / batch_size) + 1
        while True:
            # Before each epoch we shuffle the images' ids
            random.shuffle(self.images_ids_in_subset)
            for i in range(nb_batches):
                # We first get all the images' ids for the next batch
                current_bach = self.images_ids_in_subset[i*batch_size:(i+1)*batch_size]
                X_batch = []
                Y_batch = []
                for image_id in current_bach:
                    # Load the image and resize it. We get a PIL Image object 
                    img = image.load_img(os.path.join(self.images_path, image_id + EXT), grayscale=False, target_size=(IMG_HEIGHT, IMG_WIDTH))                                    
                    #treat regions instead of whole images
                    regions = self._get_regions_of_image(image_id)
                    
                    for region in regions:
                        imgPath = os.path.join('images', region[0])
                        img = image.load_img(imgPath, grayscale=False, target_size=(IMG_HEIGHT, IMG_WIDTH))       
                        # Cast the Image object to a numpy array and put the channel has the last dimension
                        img_arr = image.img_to_array(img, data_format='channels_last')
                        X_batch.append(img_arr)       
                        y = self._label_to_one_hot(region[1])
                        Y_batch.append(y)
                    
                # resize X_batch in (batch_size, IMG_HEIGHT, IMG_WIDTH, 3) 
                X_batch = np.reshape(X_batch, (-1, IMG_HEIGHT, IMG_WIDTH, 3))
                # resize Y_batch in (None, nb_classes) 
                Y_batch = np.reshape(Y_batch, (-1, self.nb_classes+1))
                # The preprocess consists of substracting the ImageNet RGB means values
                # https://github.com/keras-team/keras/blob/master/keras/applications/imagenet_utils.py#L72
                X_batch = preprocess_input(X_batch, data_format='channels_last')
                yield(X_batch, Y_batch)          

In [42]:
from keras.layers import Input, Dense
from keras.models import Model
from keras.optimizers import SGD
import numpy as np


from keras.applications.resnet50 import ResNet50
# Load ResNet50 architecture & its weights
model = ResNet50(include_top=True, weights='imagenet')
model.layers.pop()
model = Model(input=model.input,output=model.layers[-1].output)
model.compile(loss='binary_crossentropy', optimizer=SGD(0.5, momentum=0.9), metrics=['binary_accuracy'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("av...)`
  # This is added back by InteractiveShellApp.init_path()


In [43]:

DATAPATH = 'VOCdevkit/VOC2007'
data_gen = DataGen('trainval', DATAPATH,200)

data_set = data_gen.DataSet
print(data_set)

[8914 2858 5367 8461 4576 9411 8122 6455 7958  915 3484 7795 7890 9409
 2505 9324 3133 5219 2533 3061 6839 2915 9758 7436 7799  919  885 2941
 2202 7688 9709 7899 6625 8033 5873 6497 5615 5508 4111 7600 4215 8732
 5830 7129 6612 1083 6945 6738 7056 8425 4943 8859 2387 3462 6475 7048
 4017 4584 1414 5510 5314 5319 6842 2559 3565 5789 5032 6417 1360 8885
 2233 6079 1130 3983 3086 3057 2635 2439 5363 1738 3256 4625  964 8096
 6018 9833 8801 4351 6427 1544 3696 7729 9733 8931 2875 5679 3430 9504
 3063 6352 8876 6874 2670 5840 4189 6617 7445 9334 1732 6262 6247  635
 9098 8883 2373 8108 3330 4259 5423 8351 9138 2801 8177 7490 5186 9726
 3351 8105 1643 3107 5336 1192 6519  513 2253 8410 7059 8150 4551 5824
 9035  793 7649 7046 4553  888 1439 5085  980 7748 4897   20 4229 5989
  460 8268 8742 9896 4872 4484 9354 8173 8999 6858 7006 2263 3458 3735
 6699 9823 5554 3145 2171 2064 6073 3752 7779 9655 6254  263 3796 5320
 4818 9819 9797 9851 4361  355 4654 2986 4852 3635  192 8116 6241 6442
 1999 

In [45]:
data_set = data_gen.DataSet
print(len(data_set))

# Initilisation des matrices contenant les Deep Features et les labels
X_train =[] 
Y_train =[]
k=1
for i in data_set:
    # Pour chaque image, on extrait les images des regions d'entrée X et les labels y
    print("%d - Treating Image N° %06d"%(k,i))
    X,Y = data_gen.generate_xy_from_image(i)
    nb_region = len(Y)
    print("\t %d regions proposed"%nb_region)
    # On récupère les Deep Feature par appel à predict
    y_pred = model.predict(X)
    print("\t deep features extracted")
    X_train.extend(y_pred)
    Y_train.extend(Y)
    k+=1
l = len(Y_train)

X_train = np.reshape(X_train, (l, 2048))
Y_train = np.reshape(Y_train, (l, 22))

outfile = 'RP_DF_ResNet50_VOC2007_train'
np.savez(outfile, X_train=X_train, Y_train=Y_train)

200
1 - Treating Image N° 008914


/usr/local/lib/python3.6/dist-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


	 255 regions proposed
	 deep features extracted
2 - Treating Image N° 002858
	 165 regions proposed
	 deep features extracted
3 - Treating Image N° 005367
	 164 regions proposed
	 deep features extracted
4 - Treating Image N° 008461
	 261 regions proposed
	 deep features extracted
5 - Treating Image N° 004576
	 175 regions proposed
	 deep features extracted
6 - Treating Image N° 009411
	 197 regions proposed
	 deep features extracted
7 - Treating Image N° 008122
	 273 regions proposed
	 deep features extracted
8 - Treating Image N° 006455
	 107 regions proposed
	 deep features extracted
9 - Treating Image N° 007958
	 189 regions proposed
	 deep features extracted
10 - Treating Image N° 000915
	 302 regions proposed
	 deep features extracted
11 - Treating Image N° 003484
	 295 regions proposed
	 deep features extracted
12 - Treating Image N° 007795
	 265 regions proposed
	 deep features extracted
13 - Treating Image N° 007890
	 267 regions proposed
	 deep features extracted
14 - Treati

ValueError: ignored

In [0]:
X_train = np.reshape(X_train, (l, 2048))
Y_train = np.reshape(Y_train, (l, 21))

outfile = 'RP_DF_ResNet50_VOC2007_train'
np.savez(outfile, X_train=X_train, Y_train=Y_train)

In [31]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping


#reload generator
#%load_ext autoreload
#%autoreload 1
#%aimport od_data_gen

DATAPATH = 'VOCdevkit/VOC2007'
data_gen = DataGen('trainval', DATAPATH,200)


earlyStopping=EarlyStopping(monitor='val_loss', patience=150,min_delta=0.0001, verbose=0, mode='min')

#Creer le model du classifieur
classifier = Sequential()
classifier.add(Dense(22,  input_dim=2048, name='prediction', activation='sigmoid'))
classifier.summary()



#sgd= SGD(lr=0.001, momentum=0.8, decay = 0)
classifier.compile(loss='binary_crossentropy',optimizer='adam',metrics=['binary_accuracy'])
#classifier.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['categorical_accuracy'])



batch_size=128
nb_epochs=2500


#classifier.fit(X_train, Y_train,batch_size=batch_size, epochs=nb_epochs,verbose=1)


split = 3

steps_per_epoch_train =int(len(Y_train) / batch_size) + 1
classifier.fit_generator(data_gen.bias_flow(X_train,Y_train,batch_size=batch_size,full_batch = 2),
                     validation_data = (data_gen.bias_flow(X_train[split:],Y_train[split:],batch_size=batch_size,full_batch = 2)),
                    #validation_data = (X_val,Y_val),
                    validation_steps = steps_per_epoch_train,
                    callbacks=[earlyStopping],
                    steps_per_epoch=steps_per_epoch_train,
                    epochs=nb_epochs,
                    verbose=1)

scores = classifier.evaluate(X_test, Y_test, verbose=0)
print("%s TEST: %.2f%%" % (classifier.metrics_names[1], scores[1]*100))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
prediction (Dense)           (None, 22)                45078     
Total params: 45,078
Trainable params: 45,078
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2500
20/20 [==============================] - 4s 181ms/step - loss: 0.2446 - binary_accuracy: 0.9335 - val_loss: 0.0520 - val_binary_accuracy: 1.0000
Epoch 2/2500
20/20 [==============================] - 0s 8ms/step - loss: 0.0278 - binary_accuracy: 1.0000 - val_loss: 0.0150 - val_binary_accuracy: 1.0000
Epoch 3/2500
20/20 [==============================] - 0s 9ms/step - loss: 0.0116 - binary_accuracy: 1.0000 - val_loss: 0.0091 - val_binary_accuracy: 1.0000
Epoch 4/2500
20/20 [==============================] - 0s 8ms/step - loss: 0.0079 - binary_accuracy: 1.0000 - val_loss: 0.0068 - val_binary_accuracy: 1.0000
Epoch 5/2500
20/20 [====================

NameError: ignored